# Proceso de ETL en los 3 Data Set  / ETL process in the 3 Data Set

### Importamos galerias a utilizar / We import galleries to use

In [1]:
import pandas as pd
import json

## DataSet Out Put steam games


### Cargamos el archivo .json en un dataframe / We load the .json file into a dataframe

In [4]:

data = []
with open('DataSet\PI MLOps - STEAM\output_steam_games.json') as e:
    for line in e:
        data.append(json.loads(line))
dfgames = pd.DataFrame(data)



<>:3: SyntaxWarning: invalid escape sequence '\P'
<>:3: SyntaxWarning: invalid escape sequence '\P'
C:\Users\alejo\AppData\Local\Temp\ipykernel_14324\2203093654.py:3: SyntaxWarning: invalid escape sequence '\P'
  with open('DataSet\PI MLOps - STEAM\output_steam_games.json') as e:


### Borro valores faltantes  / I delete missing values

In [5]:
dfgames = dfgames.dropna() 

### Visualizacion de las columnas / Column display

In [6]:
dfgames.columns 

Index(['publisher', 'genres', 'app_name', 'title', 'url', 'release_date',
       'tags', 'reviews_url', 'specs', 'price', 'early_access', 'id',
       'developer'],
      dtype='object')

### Borro las columnas que no necesito / I delete the columns I don't need

In [7]:

dfgames = dfgames.drop(['publisher', 'reviews_url','url','tags', 'app_name', 'specs', 'price', 'early_access', 'developer'], axis=1)

### Creamos dummys para genres / We create dummies for genres

In [ ]:
dfgames['genres'] = dfgames['genres'].fillna('[]')  # Rellenar los valores faltantes con una lista vacía / fill missing values ​​with empty list
dfgames['genres'] = dfgames['genres'].apply(lambda x: ', '.join(x))  # Convertir la lista de géneros a una cadena separada por comas / Convert genre list to comma separated string


### Crear variables ficticias para los géneros / Create dummy variables for genres

In [ ]:

dummy_genres = dfgames['genres'].str.get_dummies(', ')

### Concatenar las variables ficticias con el DataFrame original / Concatenate the dummy variables with the original DataFrame 

In [ ]:

dfgames_with_dummies = pd.concat([dfgames, dummy_genres], axis=1)

In [9]:
print(dfgames_with_dummies)

                                                 genres  \
88310       Action, Casual, Indie, Simulation, Strategy   
88311                Free to Play, Indie, RPG, Strategy   
88312   Casual, Free to Play, Indie, Simulation, Sports   
88313                         Action, Adventure, Casual   
88315                     Action, Adventure, Simulation   
...                                                 ...   
120439                 Action, Adventure, Casual, Indie   
120440              Casual, Indie, Simulation, Strategy   
120441                          Casual, Indie, Strategy   
120442                        Indie, Racing, Simulation   
120443                                    Casual, Indie   

                           title release_date      id  Action  Adventure  \
88310        Lost Summoner Kitty   2018-01-04  761140       1          0   
88311                  Ironbound   2018-01-04  643980       0          0   
88312    Real Pool 3D - Poolians   2017-07-24  670290       0  

### De la columna release_date extraigo solo el año / From the release_date column I extract only the year


In [11]:
dfgames = dfgames_with_dummies
# Reemplazar los valores no válidos por NaN / Replace invalid values ​​with NaN
dfgames["release_date"] = pd.to_datetime(dfgames["release_date"], errors='coerce')
# Filtrar el DataFrame para obtener los registros válidos / Filter the DataFrame to get valid records
df_filtered = dfgames.dropna(subset=["release_date"])
# Convertir la columna "release_date" a tipo datetime / Convert "release_date" column to datetime type
df_filtered["release_date"] = pd.to_datetime(df_filtered["release_date"])
# Rellenando los valores faltantes en las columna'release_date' con la moda / Filling missing values ​​in 'release_date' columns with mode
columns_fill_mode = ['release_date']
for col in columns_fill_mode:
    dfgames[col].fillna(dfgames[col].mode()[0], inplace=True)
# Modelado y eliminación de la columna 'release_date' / Modeling and removing 'release_date' column
dfgames['year'] = dfgames['release_date'].dt.year.astype('Int64')
dfgames.drop('release_date', axis=1, inplace=True)

C:\Users\alejo\AppData\Local\Temp\ipykernel_14324\2930503749.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_filtered["release_date"] = pd.to_datetime(df_filtered["release_date"])


In [12]:
dfgames

,genres,title,id,Action,Adventure,Animation &amp; Modeling,Audio Production,Casual,Design &amp; Illustration,Early Access,...,RPG,Racing,Simulation,Software Training,Sports,Strategy,Utilities,Video Production,Web Publishing,year
88310,"Action, Casual, Indie, Simulation, Strategy",Lost Summoner Kitty,761140,1,0,0,0,1,0,0,...,0,0,1,0,0,1,0,0,0,2018
88311,"Free to Play, Indie, RPG, Strategy",Ironbound,643980,0,0,0,0,0,0,0,...,1,0,0,0,0,1,0,0,0,2018
88312,"Casual, Free to Play, Indie, Simulation, Sports",Real Pool 3D - Poolians,670290,0,0,0,0,1,0,0,...,0,0,1,0,1,0,0,0,0,2017
88313,"Action, Adventure, Casual",弹炸人2222,767400,1,1,0,0,1,0,0,...,0,0,0,0,0,0,0,0,0,2017
88315,"Action, Adventure, Simulation",Battle Royale Trainer,772540,1,1,0,0,0,0,0,...,0,0,1,0,0,0,0,0,0,2018
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
120439,"Action, Adventure, Casual, Indie",Kebab it Up!,745400,1,1,0,0,1,0,0,...,0,0,0,0,0,0,0,0,0,2018
120440,"Casual, Indie, Simulation, Strategy",Colony On Mars,773640,0,0,0,0,1,0,0,...,0,0,1,0,0,1,0,0,0,2018
120441,"Casual, Indie, Strategy",LOGistICAL: South Africa,733530,0,0,0,0,1,0,0,...,0,0,0,0,0,1,0,0,0,2018
120442,"Indie, Racing, Simulation",Russian Roads,610660,0,0,0,0,0,0,0,...,0,1,1,0,0,0,0,0,0,2018


### Importamos a csv / We import to csv

In [13]:
dfgames.to_csv('output_steam_games_clean.csv') 

## DataSet australian user reviews 

### Cargamos el archivo .json en un dataframe / We load the .json file into a dataframe

In [18]:
with open('australian_user_reviews.json', 'r', encoding='utf-8') as w:
    data = w.readlines()
    
datos = [eval(line.strip()) for line in data]

df_review = pd.DataFrame(datos)
df_review.head()

,user_id,user_url,reviews
0,76561197970982479,http://steamcommunity.com/profiles/76561197970...,"[{'funny': '', 'posted': 'Posted November 5, 2..."
1,js41637,http://steamcommunity.com/id/js41637,"[{'funny': '', 'posted': 'Posted June 24, 2014..."
2,evcentric,http://steamcommunity.com/id/evcentric,"[{'funny': '', 'posted': 'Posted February 3.',..."
3,doctr,http://steamcommunity.com/id/doctr,"[{'funny': '', 'posted': 'Posted October 14, 2..."
4,maplemage,http://steamcommunity.com/id/maplemage,"[{'funny': '3 people found this review funny',..."


### Desanidamos la columna reviews / We unnested the reviews column

In [19]:
df_reviewsOpen = df_review.explode('reviews') 
dfreviewsOpen = pd.concat([df_reviewsOpen.drop(['reviews'], axis=1), df_reviewsOpen['reviews'].apply(pd.Series)], axis=1) 

### Creo la columna year / I create the year column

In [ ]:
dfreviewsOpen['posted year'] = dfreviewsOpen['posted'].str.extract(r'(\d{4})')

### Elimino la columna posted / I delete the posted column

In [ ]:
dfreviewsOpen.drop('posted' , axis = 1, inplace = True)

### Previsualizamos / We preview 

In [21]:
dfreviewsOpen.head(3)

,user_id,user_url,funny,last_edited,item_id,helpful,recommend,review,0,posted year
0,76561197970982479,http://steamcommunity.com/profiles/76561197970...,,,1250,No ratings yet,True,Simple yet with great replayability. In my opi...,NaN,2011
0,76561197970982479,http://steamcommunity.com/profiles/76561197970...,,,22200,No ratings yet,True,It's unique and worth a playthrough.,NaN,2011
0,76561197970982479,http://steamcommunity.com/profiles/76561197970...,,,43110,No ratings yet,True,Great atmosphere. The gunplay can be a bit chu...,NaN,2011


### Realizamos el Sentiment analyzer con la galeria nlkt / We carry out the Sentiment analyzer with the nlkt gallery

In [23]:
import nltk
nltk.download('vader_lexicon')
from nltk.sentiment.vader import SentimentIntensityAnalyzer

# Inicializar el analizador de sentimientos / Initialize the sentiment analyzer
sia = SentimentIntensityAnalyzer()

# Función para asignar valores según la escala / Function to assign values according to the scale
def get_sentiment_score(text):
    if pd.isnull(text) or text == '':
        return 1  # Return neutral if it is empty or NaN
    elif isinstance(text, str):
        sentiment = sia.polarity_scores(text)
        compound_score = sentiment['compound']
        if compound_score >= -0.05:
            return 0  # Good score
        elif compound_score <= -0.05:
            return 2  # Bad score
        else:
            return 1
    else:
        return 1  # Return neutral for non-string values


# Convertir la columna 'review' a tipo de dato str / Convertir la columna 'review' a tipo de dato str
dfreviewsOpen['review'] = dfreviewsOpen['review'].astype(str)

# Aplicar la función get_sentiment_score a la columna 'review' / Apply the get_sentiment_score function to the 'review' column
dfreviewsOpen['sentiment_score'] = dfreviewsOpen['review'].apply(get_sentiment_score)

# Imprimir el DataFrame actualizado / Print the updated DataFrame
print(dfreviewsOpen)

[nltk_data] Downloading package vader_lexicon to
[nltk_data]     C:\Users\alejo\AppData\Roaming\nltk_data...


                 user_id                                           user_url  \
0      76561197970982479  http://steamcommunity.com/profiles/76561197970...   
0      76561197970982479  http://steamcommunity.com/profiles/76561197970...   
0      76561197970982479  http://steamcommunity.com/profiles/76561197970...   
1                js41637               http://steamcommunity.com/id/js41637   
1                js41637               http://steamcommunity.com/id/js41637   
...                  ...                                                ...   
25797  76561198312638244  http://steamcommunity.com/profiles/76561198312...   
25797  76561198312638244  http://steamcommunity.com/profiles/76561198312...   
25798        LydiaMorley           http://steamcommunity.com/id/LydiaMorley   
25798        LydiaMorley           http://steamcommunity.com/id/LydiaMorley   
25798        LydiaMorley           http://steamcommunity.com/id/LydiaMorley   

                                  funny last_edited

In [24]:
dfreviewsOpen.to_csv('australian_user_reviews_clean.csv') 

In [25]:
dfreviewsOpen.head(3)

,user_id,user_url,funny,last_edited,item_id,helpful,recommend,review,0,posted year,sentiment_score
0,76561197970982479,http://steamcommunity.com/profiles/76561197970...,,,1250,No ratings yet,True,Simple yet with great replayability. In my opi...,NaN,2011,0
0,76561197970982479,http://steamcommunity.com/profiles/76561197970...,,,22200,No ratings yet,True,It's unique and worth a playthrough.,NaN,2011,0
0,76561197970982479,http://steamcommunity.com/profiles/76561197970...,,,43110,No ratings yet,True,Great atmosphere. The gunplay can be a bit chu...,NaN,2011,0


## DataSet australian user items 

### Leemos el archivo .json / We read the .json file

In [27]:
with open("australian_users_items.json", 'r', encoding='utf-8') as i:
    data = i.readlines()
    
records = [eval(line.strip()) for line in data]

df_item = pd.DataFrame(records)
df_item.head()

,user_id,items_count,steam_id,user_url,items
0,76561197970982479,277,76561197970982479,http://steamcommunity.com/profiles/76561197970...,"[{'item_id': '10', 'item_name': 'Counter-Strik..."
1,js41637,888,76561198035864385,http://steamcommunity.com/id/js41637,"[{'item_id': '10', 'item_name': 'Counter-Strik..."
2,evcentric,137,76561198007712555,http://steamcommunity.com/id/evcentric,"[{'item_id': '1200', 'item_name': 'Red Orchest..."
3,Riot-Punch,328,76561197963445855,http://steamcommunity.com/id/Riot-Punch,"[{'item_id': '10', 'item_name': 'Counter-Strik..."
4,doctr,541,76561198002099482,http://steamcommunity.com/id/doctr,"[{'item_id': '300', 'item_name': 'Day of Defea..."


### Aplico la funcion lambda para desanidar las keys y sus valores / I apply the lambda function to unnest the keys and their values

In [28]:

df_item['playtime_forever'] = df_item['items'].apply(lambda x: x[0].get('playtime_forever') if len(x) > 0 else 0)
df_item['playtime_2weeks'] = df_item['items'].apply(lambda x: x[0].get('playtime_2weeks') if len(x) > 0 else 0)
df_item['id'] = df_item['items'].apply(lambda x: x[0].get('item_id') if len(x) > 0 else 0)
df_item['playtime_forever'] = df_item['playtime_forever'].astype(int)
df_item['playtime_2weeks'] = df_item['playtime_2weeks'].astype(int)

### Elimino columnas que no usare / I eliminate columns that I will not use

In [29]:
df_item.drop(['user_url'], axis=1, inplace=True)  
df_item.drop(['items'], axis=1, inplace=True)

In [30]:
df_item

,user_id,items_count,steam_id,playtime_forever,playtime_2weeks,id
0,76561197970982479,277,76561197970982479,6,0,10
1,js41637,888,76561198035864385,0,0,10
2,evcentric,137,76561198007712555,923,0,1200
3,Riot-Punch,328,76561197963445855,0,0,10
4,doctr,541,76561198002099482,1131,0,300
...,...,...,...,...,...,...
88305,76561198323066619,22,76561198323066619,0,0,413850
88306,76561198326700687,177,76561198326700687,0,0,11020
88307,XxLaughingJackClown77xX,0,76561198328759259,0,0,0
88308,76561198329548331,7,76561198329548331,677,677,304930


### Exportamos en formato csv / We export in csv format

In [31]:
df_item.to_csv('australian_users_items_clean.csv', index=False)

## Creacion de un unico dataset / Creation of a single dataset

### Unimos tablas con id / We join tables with id

In [32]:

merged_df = pd.merge(dfgames, df_item, on='id') 

### Borramos genres / We delete genres

In [33]:
merged_df.drop(['genres'],axis=1, inplace=True) 

### Guardamos en csv / We save in csv

In [34]:
merged_df.to_csv('Df_Final.csv', index=False) 

### Cargamos reviews / We load reviews

In [35]:
df_review = pd.read_csv('australian_user_reviews_clean.csv') 

### Agruego reviews al merge anterior / I add reviews to the previous merge

In [36]:
df_final = pd.merge(merged_df, df_review, on = 'user_id') 

In [37]:
df_final.columns

Index(['title', 'id', 'Action', 'Adventure', 'Animation &amp; Modeling',
       'Audio Production', 'Casual', 'Design &amp; Illustration',
       'Early Access', 'Education', 'Free to Play', 'Indie',
       'Massively Multiplayer', 'Photo Editing', 'RPG', 'Racing', 'Simulation',
       'Software Training', 'Sports', 'Strategy', 'Utilities',
       'Video Production', 'Web Publishing', 'year', 'user_id', 'items_count',
       'steam_id', 'playtime_forever', 'playtime_2weeks', 'Unnamed: 0',
       'user_url', 'funny', 'last_edited', 'item_id', 'helpful', 'recommend',
       'review', '0', 'posted year', 'sentiment_score'],
      dtype='object')

### Borramos otras columnas que no vamos a utilizar / We delete other columns that we are not going to use

In [38]:
df_final.drop(columns=['0', 'Unnamed: 0'], inplace=True)

### Exportamos a csv / We export to csv

In [39]:
df_final.to_csv('DataSet_Final.csv', index=False)

### Creamos otro DataSet pero sin las columnas genre para utilizarlo en algunos procesos del EDA / We create another DataSet but without the genre columns to use it in some EDA processes  

In [40]:
df_num= df_final.select_dtypes(include = (int,float))
df_num.drop(columns=['Action', 'Adventure',
       'Audio Production', 'Casual', 'Design &amp; Illustration',
       'Early Access', 'Education', 'Free to Play', 'Indie', 'Animation &amp; Modeling',
       'Massively Multiplayer', 'Photo Editing', 'RPG', 'Racing', 'Simulation',
       'Software Training', 'Sports', 'Strategy', 'Utilities',
       'Video Production', 'Web Publishing'], inplace=True)

### Exportamos a csv / export to csv

In [42]:
df_num.to_csv('DataSet_Final_Numerico.csv', index=False)